In [6]:
#多线程爬取信息
import multiprocessing as mp;
import time;
from urllib.request import urlopen,urljoin;
from bs4 import BeautifulSoup;
import re;
base_url = "https://morvanzhou.github.io/";
#定义两个功能 一个是用来进行爬取的crawl 一个是用来解析的parse
def crawl(url):
    response = urlopen(url);
    return response.read().decode();
def parse(html):
    soup = BeautifulSoup(html,'lxml');
    urls = soup.find_all('a',{"href":re.compile('^/.+?/$')});
    title = soup.find('h1').get_text().strip();
    #去重
    page_urls = set([urljoin(base_url,url['href']) for url in urls]);
    url = soup.find('meta',{'property':'og:url'})['content'];
    return title,page_urls,url;
#定义两个set 用来搜集爬过的网页和没有爬过的
unseen = set([base_url,]);
seen = set();


In [7]:
#测试单进程的进行爬虫
if base_url != "https://morvanzhou.github.io/":
    restricted_crawl = True
else:
    restricted_crawl = False
while len(unseen) != 0:
    if restricted_crawl and len(seen) >= 20:
        break;
    htmls = [crawl(url) for url in unseen];
    results = [parse(html) for html in htmls];
    seen.update(unseen);
    unseen.clear();
    for title,page_urls,url in results:
        unseen.update(page_urls - seen);
        

In [8]:
#测试分布式的进行爬虫
pool = mp.Pool(4);
while len(unseen) != 0:
    crawl_jobs = [pool.apply_async(crawl,args = (url,)) for url in unseen];
    htmls = [j.get() for j in crawl_jobs];
    parse_jobs = [pool.apply_async(parse,args = (html,)) for html in htmls];
    results = [j.get() for j in parse_jobs];
    